In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

In [40]:
df = pd.read_csv('results.csv')

nn_df = df[df['name'] == 'nn'].reset_index()
gc_df = df[df['name'] == 'gc'].reset_index()
ls_random_df = df[df['name'] == 'ls-random'].reset_index()
ls_nn_df = df[df['name'] == 'ls-nn'].reset_index()
ils_df = df[df['name'] == 'ils'].reset_index()

nn_df.head(n=5)

,index,name,primary_graph,secondary_graph,time_ns,primary_cost,secondary_cost,total_cost
0,100,nn,15-87-21-93-69-52-78-17-23-37-35-98-83-9-71-20...,70-40-99-47-13-2-45-28-33-82-54-11-26-85-34-19...,10057143,13228,17885,31113
1,101,nn,28-45-2-42-13-70-40-99-47-51-77-95-4-36-32-75-...,93-21-15-87-52-78-17-23-37-35-98-83-9-71-20-73...,15367143,12480,13916,26396
2,102,nn,74-18-52-78-17-23-37-35-98-83-9-71-20-73-58-16...,77-51-95-4-36-32-75-12-94-81-49-43-72-67-84-38...,8882314,13541,12953,26494
3,103,nn,75-32-12-94-81-49-43-72-67-84-38-29-95-77-51-4...,9-83-71-20-73-58-16-14-10-31-90-97-22-44-46-62...,4580055,12707,18463,31170
4,104,nn,57-27-92-66-0-62-5-48-89-9-83-71-20-73-58-16-1...,6-8-56-86-50-60-24-80-68-72-49-43-1-53-39-63-6...,3417160,13603,12502,26105


In [61]:
def parse_graph():
    pass

def get_data(solver_df):
    tc = solver_df['total_cost']
    best_cost = solver_df.iloc[tc.idxmin('column')]['total_cost']
    worst_cost = solver_df.iloc[tc.idxmax('column')]['total_cost']
    avg_cost = tc.mean()
    
    time = solver_df['time_ns']
    best_time = solver_df.iloc[time.idxmin('column')]['time_ns'] / 1000000
    worst_time = solver_df.iloc[time.idxmax('column')]['time_ns'] / 1000000
    avg_time = time.mean() / 1000000
    
    best_primary_graph = solver_df.iloc[tc.idxmin('column')]['primary_graph']
    best_secondary_graph = solver_df.iloc[tc.idxmin('column')]['secondary_graph']

    return {
        'best_result': {
            'primary': best_primary_graph,
            'secondary': best_secondary_graph
        },
        'best_cost': best_cost,
        'worst_cost': worst_cost,
        'avg_cost': avg_cost,
        'best_time': best_time,
        'worst_time': worst_time,
        'avg_time': avg_time
    }

In [62]:
get_data(ls_random_df)

{'best_result': {'primary': '60-24-86-84-38-95-77-47-99-70-13-2-42-26-85-41-55-96-74-3-69-21-15-87-17-78-48-5-0-92-14-16-58-73-20-61-56-6-29-32-75-12-94-63-35-23-83-9-27-57',
  'secondary': '91-62-46-66-43-1-39-53-88-25-64-65-93-52-89-71-31-76-19-8-50-67-51-4-36-81-49-72-68-80-28-40-45-33-82-97-90-22-59-11-54-34-44-10-98-37-18-79-30-7'},
 'best_cost': 47692,
 'worst_cost': 63822,
 'avg_cost': 54821.22,
 'best_time': 49.053852,
 'worst_time': 262.467574,
 'avg_time': 86.99462616}

In [76]:
COLUMNS_COST = ['min_cost', 'max_cost', 'avg_cost']
COLUMNS_TIME = ['min_time', 'max_time', 'avg_time']

def to_series(data_dict, key, columns):
    series = []
    indexes = []
    
    for solver, values in data_dict.items():
        best = values['best_{}'.format(key)]
        worst = values['worst_{}'.format(key)]
        avg = values['avg_{}'.format(key)]
        
        indexes.append(solver)
        series.append(pd.Series([best, worst, avg], index=columns))
    
    return pd.DataFrame(series, index=indexes)

results = {
    'nn': get_data(nn_df),
    'gc': get_data(gc_df),
    'ls_random': get_data(ls_random_df),
    'ls_nn': get_data(ls_nn_df),
    'ils': get_data(ils_df)
#     'ea': get_data(nn_df)
}

In [77]:
to_series(results, 'cost', COLUMNS_COST)

,min_cost,max_cost,avg_cost
nn,25201.0,33275.0,29491.01
gc,28492.0,36054.0,32152.95
ls_random,47692.0,63822.0,54821.22
ls_nn,47754.0,47754.0,47754.00
ils,44516.0,50788.0,47820.27


In [78]:
to_series(results, 'time', COLUMNS_TIME)

,min_time,max_time,avg_time
nn,1.321579,19.201133,3.397330
gc,3.809146,118.500420,9.948993
ls_random,49.053852,262.467574,86.994626
ls_nn,83.238218,196.560560,115.902864
ils,7233.045487,13273.879027,8775.415420
